# Chapter 4
Christoph Windheuser    
April 15, 2022   
Python examples of chapter 3 in the book *Learning Spark*



In [1]:
# Import required python spark libraries
import findspark
import pyspark

from pyspark.sql.types import *
from pyspark.sql.functions import col, expr, when, concat, lit, avg, desc
from pyspark.sql import SparkSession
from pyspark.sql import Row


In [2]:
# Connect Jupyter Notebook with the Spark application and create Spark Context
findspark.init()
sc = pyspark.SparkContext(appName="chapter_4")


In [3]:
#create a SparkSession
spark = (SparkSession
       .builder
       .appName("Chapter_4_Examples")
       .getOrCreate())


In [4]:
csv_file = "data/departuredelays.csv"

df = (spark.read.format("csv")
      .option("inferSchema", "true")
      .option("header", "true")
      .load(csv_file))

df.createOrReplaceTempView("us_delay_flights_tbl")


In [5]:
df.show(5)

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
+-------+-----+--------+------+-----------+
only showing top 5 rows



Show flights with a distance of > 1000 miles and order the results by descendent distance. Show the first 10 results of this list:

In [6]:
spark.sql("""SELECT distance, origin, destination
          FROM us_delay_flights_tbl WHERE distance > 1000
          ORDER BY distance DESC""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



Instead of spark.sql, the same querry can be executed with the DataFrame API and shows the same result: 

In [7]:
(df.select("distance", "origin", "destination")
   .where("distance > 1000")
   .orderBy("distance", ascending = False).show(10))

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



Find all flights between San Francisco (SFO) and Chicago (ORD) with at least a two-hour delay:

In [8]:
spark.sql("""SELECT date, delay, origin, destination
          FROM us_delay_flights_tbl
          WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD'
          ORDER by delay DESC""").show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows



Label the flights based on the delays they have experienced. Add a human-readable new column called 'Flight_Delays' containing the labels to the table:

In [9]:
spark.sql("""SELECT delay, origin, destination,
          CASE
              WHEN delay > 360 THEN 'Very Long Delays'
              WHEN delay > 120 AND delay < 360 THEN 'Long Delay'
              WHEN delay > 60 AND delay < 120 THEN 'Short Delay'
              WHEN delay > 0 AND delay < 60 THEN 'Tolerable Delay'
              WHEN delay = 0 THEN 'No Delay'
              ELSE 'Early'
         END AS Flight_Delays
         FROM us_delay_flights_tbl
         ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|   Long Delay|
|  305|   ABE|        ATL|   Long Delay|
|  275|   ABE|        ATL|   Long Delay|
|  257|   ABE|        ATL|   Long Delay|
|  247|   ABE|        ATL|   Long Delay|
|  247|   ABE|        DTW|   Long Delay|
|  219|   ABE|        ORD|   Long Delay|
|  211|   ABE|        ATL|   Long Delay|
|  197|   ABE|        DTW|   Long Delay|
|  192|   ABE|        ORD|   Long Delay|
+-----+------+-----------+-------------+
only showing top 10 rows



## Creating SQL Tables and Views
(Chapter 4, page 89 ff)

### Create a database called `learn_spark_db`

In [121]:
spark.sql("CREATE DATABASE learn_spark_db")

DataFrame[]

In [122]:
spark.sql("USE learn_spark_db")

DataFrame[]

Spark creates a dictionary `learn_spark_db.db` in the "spark.sql.warehouse.dir" to save tables of the new database.     
The "spark.sql.warehouse.dir" can be get by:

In [123]:
print(spark.conf.get("spark.sql.warehouse.dir"))

file:/home/christoph/Dev/LearningSpark/spark-warehouse


### Create a managed table

Create a managed table with the SQL API:

In [124]:
spark.sql("""CREATE TABLE managed_us_delay_flights_tbl
             (date STRING, delay INT, distance INT, origin STRING, destination STRING)
              USING CSV OPTIONS (PATH '/home/christoph/Dev/LearningSpark/data/departuredelays.csv')""")


DataFrame[]

In [146]:
spark.sql("SELECT * FROM managed_us_delay_flights_tbl").show(10)


AnalysisException: Table or view not found: managed_us_delay_flights_tbl; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [managed_us_delay_flights_tbl], [], false


Let's drop the table and create it again with the DataFrame API:

In [147]:
spark.sql("DROP TABLE managed_us_delay_flights_tbl")

AnalysisException: Table or view not found: managed_us_delay_flights_tbl; line 1 pos 11;
'DropTable false, false
+- 'UnresolvedTableOrView [managed_us_delay_flights_tbl], DROP TABLE, true


In [148]:
csv_file = "data/departuredelays.csv"
schema = "date STRING, delay INT, distance INT, origin STRING, destination STRING"
flights_df = spark.read.csv(csv_file, schema = schema)


In [149]:
flights_df.show(3)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|    date| null|    null|origin|destination|
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
+--------+-----+--------+------+-----------+
only showing top 3 rows



In [150]:
flights_df.write.saveAsTable("managed_us_delay_flights_tbl")

In [151]:
spark.sql("SELECT * FROM managed_us_delay_flights_tbl").show(3)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01201755|    0|     449|   ORF|        ATL|
|01201610|   52|     449|   ORF|        ATL|
|01201441|    0|     449|   ORF|        ATL|
+--------+-----+--------+------+-----------+
only showing top 3 rows



Drop the table again:

In [152]:
spark.sql("DROP TABLE managed_us_delay_flights_tbl")

DataFrame[]

At the end, let's drop the database `learn_spark_db`and all the tables in this database:

In [132]:
spark.sql("DROP DATABASE learn_spark_db CASCADE")

DataFrame[]

### Creating an unmanaged table

Let's first create and use a database:

In [133]:
spark.sql("CREATE DATABASE learn_spark_db")

DataFrame[]

In [134]:
spark.sql("USE learn_spark_db")

DataFrame[]

Now let's create the table `us_delay_flights_tbl` with an SQL command:

In [155]:
spark.sql("""CREATE TABLE us_delay_flights_tbl
             (date STRING, delay INT, distance INT, origin STRING, destination STRING)
             USING CSV OPTIONS (PATH '/home/christoph/Dev/LearningSpark/data/departuredelays.csv')""")


DataFrame[]

In [156]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(3)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|    date| null|    null|origin|destination|
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
+--------+-----+--------+------+-----------+
only showing top 3 rows



Let's drop the table again:

In [157]:
spark.sql("DROP TABLE us_delay_flights_tbl")

DataFrame[]

Now creating the same table with the DataFrame API:

In [162]:
(flights_df
 .write
 .option("path", "/home/christoph/Dev/LearningSpark/data/departuredelays.csv")
 .saveAsTable("us_delay_flights_tbl")
)

Py4JJavaError: An error occurred while calling o675.saveAsTable.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.datasources.DataSource.writeAndRead(DataSource.scala:537)
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.saveDataIntoTable(createDataSourceTables.scala:228)
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.run(createDataSourceTables.scala:182)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:1

In [159]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(3)

AnalysisException: Table or view not found: us_delay_flights_tbl; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [us_delay_flights_tbl], [], false


And dropping it again:

In [160]:
spark.sql("DROP TABLE us_delay_flights_tbl")

AnalysisException: Table or view not found: us_delay_flights_tbl; line 1 pos 11;
'DropTable false, false
+- 'UnresolvedTableOrView [us_delay_flights_tbl], DROP TABLE, true


And at the end, let's drop the whole database:

In [120]:
spark.sql("DROP DATABASE learn_spark_db CASCADE")

DataFrame[]